In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import urllib

# Display Data about All Wildfire Incidents in Indonesia
Gather information for all wildfire events in Sumatra

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [3]:
if os.getcwd().split("\\")[-1] != "wildfire-data-science":
    os.chdir("..")
wf_data  = pd.read_csv("data-total.csv")
wf_data.columns
# rename columns
wf_data.rename(
    columns={
        'Lat': 'lat', 
        'Long': 'long', 
        'Satellite': 'satellite', 
        'Time_UTC': 'time_utc', 
        'Date': 'date', 
        'Source': 'source', 
        'PROVINSI': 'provinsi',
        'KAB_KOTA': 'kab_kota'
        }, 
    inplace=True,
    )

# adding '-' to date column values
wf_data.date = pd.to_datetime(wf_data.date, format="%Y%m%d")
wf_data.date = wf_data.date.astype('string')

In [4]:
# All provinces in sumatera
sumatera_provinces = [
    'SUMATERA UTARA',
    'JAMBI',
    'RIAU',
    'SUMATERA BARAT',
    'SUMATERA SELATAN',
    'BENGKULU',
    'LAMPUNG',
    'ACEH'
]
# create filter so that only events in sumatera will be returned.
sumatera_filter = wf_data['provinsi'].apply(lambda x: x in sumatera_provinces) 
sumatera_only = wf_data.loc[sumatera_filter]

In [5]:
display(sumatera_only.info())
display(sumatera_only.provinsi.unique())
display(sumatera_only.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7839 entries, 1 to 39670
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   lat        7839 non-null   float64
 1   long       7839 non-null   float64
 2   satellite  7839 non-null   object 
 3   time_utc   7839 non-null   object 
 4   date       7839 non-null   string 
 5   source     7839 non-null   object 
 6   provinsi   7839 non-null   object 
 7   kab_kota   7839 non-null   object 
 8   kec2006    7830 non-null   object 
 9   desaa2006  7830 non-null   object 
 10  nama_kaw   930 non-null    object 
dtypes: float64(2), object(8), string(1)
memory usage: 734.9+ KB


None

array(['LAMPUNG', 'RIAU', 'SUMATERA UTARA', 'SUMATERA BARAT',
       'SUMATERA SELATAN', 'ACEH', 'JAMBI', 'BENGKULU'], dtype=object)

,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw
1,-5.06259,105.101,LPN-NPP,06:11:04 AM,2016-08-24,LAPAN,LAMPUNG,LAMPUNG TENGAH,ANAK TUHA,BUMI JAYA,NaN
4,1.18991,100.672,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HULU,KECAMATAN KAPENUHAN,UPT SP IV KOTO TENGAH,NaN
5,2.31418,100.288,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,SUMATERA UTARA,LABUHANBATU,PANAI TENGAH,SELAT BETING,NaN
6,1.41833,100.752,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,SIARANG ARANG,NaN
8,1.38958,100.673,LPN-NPP,06:36:36 AM,2016-09-13,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,TELUK NAYANG,NaN


In [6]:
# changing the date data type to datetime
# Adding date day, month, and year to the data frame

year =[]
month=[]
day  =[]


if 'year' not in sumatera_only.columns.values:
    date_formatted = sumatera_only['date'].astype('datetime64[ns]')
    date = pd.DatetimeIndex(sumatera_only['date'])
    sumatera_only.insert(len(sumatera_only.columns), "year", date.year, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "month", date.month, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "day", date.day, True) # adding column


In [7]:
sumatera_only = sumatera_only.sort_values(by=['date'], 
                                          ascending=True,
                                          ignore_index=True)
sumatera_only


,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw,year,month,day
0,1.649680,100.892000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
1,1.648530,100.884000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
2,1.580030,100.855000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN TANAH PUTIH,BALAM SEMPURNA,NaN,2016,4,12
3,4.272360,97.747800,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,ACEH,ACEH TAMIANG,TAMIANG HULU,BATU BEDULANG,NaN,2016,4,12
4,1.656640,100.891000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7834,-4.437215,104.505066,LPN-NPP,12:44:37 PM,2020-09-16,LAPAN,LAMPUNG,WAYKANAN,BLAMBANGAN UMPU,TANJUNG RAYA GIHAM,NaN,2020,9,16
7835,0.785336,99.191719,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,MANDAILINGNATAL,NATAL,PARDAMEAN BARU,NaN,2020,10,17
7836,-0.470491,100.017670,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA BARAT,PADANGPARIAMAN,BATANG GASAN,MALAI BAWAH,NaN,2020,10,17
7837,1.306781,99.927521,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,PADANG LAWAS,BARUMUN TENGAH,SIPAGABU,NaN,2020,10,17


# Import earth engine API which is called ee


In [8]:
import ee
import folium
from folium import plugins

In [9]:
# Authenticate earth engine servers
ee.Authenticate()

Enter verification code: 4/1AX4XfWh0eB6ZCXPcgBVSofEGoWjLWkOWUrSLdO7mgdPGlV-fBYkpfvn5uic

Successfully saved authorization token.


In [10]:
#initialize API
ee.Initialize()

# 7 Januari 2022 (Download data)

In [2]:
# # Download each band individually
# def download_from_url_link(url_list, folder_path):
#     """
#     Download file using based on array of urls from parameter url_list
#     """
#     for i in range(len(url_list)):
#         file_name = str(i)
#         file_path = os.path.join(folder_path, file_name)

#         if not os.path.exists(file_path):
#             print("Downloading...", file_name)
#             urllib.request.urlretrieve(url_list[i], file_path)

#     print('done')

# sentinel 2 cloud masking 
def s2CloudMasking(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)\
                .divide(10000)\

# def download_image(download_url, folder_path, file_name):
#     download_path = os.path.join(folder_path, file_name)
#     print(f"download data from {download_path}...")
#     urllib.request.urlretrieve(download_url, download_path)
#     print(f"{download_path} downloaded")


def download_sentinel_satellite_image(
        satellite_name, situation, 
        dataset, scale, file_format, 
        folder_path):
    
    url_list = []
    
    for row in range(len(dataset)):
        file_path = os.path.join(folder_path, str(row))
        if os.path.exists(file_path):
            print("data already exist")
        else:
            os.mkdir(file_path)
            
        os.chdir(file_path)
        print(os.getcwd())
        # Set coordinates
        coordinates = [float(dataset.lat.values[row]), float(dataset.long.values[row])]
        region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                        coordinates[1]+0.2, coordinates[0]+0.2]);


        # filter image collection
        # set base date
        base_date = ee.Date(dataset.date.values[row])

        # set geometry point
        point = ee.Geometry.Point(dataset.long.values[row], dataset.lat.values[row])

        # pre fire
        prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
        prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

        # sentinel image pre wildfire event
        prefire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(prefire_date_start, prefire_date_end)\
                                   .filterBounds(region)\
                                   .map(s2CloudMasking)\
                                   .median()

        # Calculate NBR for pre fire
        pre_nbr = prefire_collection.normalizedDifference(['B8', 'B12'])


        # post fire
        postfire_date_start = ee.Date(base_date.advance(1, 'day'))
        postfire_date_end = ee.Date(base_date.advance(365, 'day'))

        # sentinel image post wildfire event
        postfire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(postfire_date_start, postfire_date_end)\
                                   .filterBounds(region)\
                                   .map(s2CloudMasking)\
                                   .median()

        # Calculate NBR for post fire
        post_nbr = postfire_collection.normalizedDifference(['B8', 'B12'])

        # calculate delta NBR
        delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
        postfire_collection = postfire_collection.addBands(delta_nbr)
        
        if situation == 'prefire':
            filenames= ['b12_b8','b4_b3','b2']
            bands =  [
                ['B12', 'B8'], 
                ['B4', 'B3'], 
                ['B2'],
                ]
            
            for i in range(len(filenames)):
                if filenames[i] not in os.listdir():
                    download_url = postfire_collection.getDownloadURL({
                        'region': region,
                        'bands' : bands[i],
                        'scale' : scale,
                        'format': file_format
                        }
                    )
                    
                    print(f"download data from {download_url}")
                    urllib.request.urlretrieve(download_url, filenames[i])
                    print(f"{download_url} downloaded")
                
                else:
                    print(f"{bands[i]} already exist")
                    
        else:
            filenames= ['dnbr_b12','b8_b4','b3_b2']
            bands =  [
                ['DELTA_NBR', 'B12'], 
                ['B8', 'B4'], 
                ['B3', 'B2'],
                ]

            for i in range(len(filenames)):
                if filenames[i] not in os.listdir():
                    download_url = postfire_collection.getDownloadURL({
                        'region': region,
                        'bands' : bands[i],
                        'scale' : scale,
                        'format': file_format
                        }
                    )

                    print(f"download data from {download_url}")
                    urllib.request.urlretrieve(download_url, filenames[i])
                    print(f"{download_url} downloaded")
                else:
                    print(f"{bands[i]} already exist")

In [15]:
# create folder fot .tiff files
D_DIR = "F:\\"

WILDFIRE_SUMATERA_DATASET_DIR = "wildfire-sumatera-dataset"
WILDFIRE_SUMATERA_GEOTIFF_DIR = "wildfire-sumatera-geotiff"

SENTINEL_DIR = "sentinel-2"

SENTINEL_PREFIRE_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{SENTINEL_DIR}\\prefire"
SENTINEL_POSTFIRE_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{SENTINEL_DIR}\\postfire"

if D_DIR is not os.getcwd():
    os.chdir(D_DIR)
    print(f"OS directory is changed to {D_DIR}")
    print(os.listdir())
    print(os.getcwd())
    
if WILDFIRE_SUMATERA_DATASET_DIR not in os.listdir():
    os.mkdir(WILDFIRE_SUMATERA_DATASET_DIR)
    print(f"{WILDFIRE_SUMATERA_DATASET_DIR} has been created")
else:
    os.chdir(D_DIR+WILDFIRE_SUMATERA_DATASET_DIR)
    print(f"{WILDFIRE_SUMATERA_DATASET_DIR} already exist")


OS directory is changed to F:\
['System Volume Information', 'Install SanDisk Software for Windows.exe', 'Install SanDisk Software for Mac.dmg', '$RECYCLE.BIN', 'wildfire-sumatera-dataset']
F:\
wildfire-sumatera-dataset already exist


In [16]:
# check whether the folder is exists
if not os.path.exists(SENTINEL_PREFIRE_FOLDER):
    os.makedirs(SENTINEL_PREFIRE_FOLDER)
    print(f"{SENTINEL_PREFIRE_FOLDER} has been successfully create")
else:
    print(f"{SENTINEL_PREFIRE_FOLDER} already exist")

if not os.path.exists(SENTINEL_POSTFIRE_FOLDER):
    os.makedirs(SENTINEL_POSTFIRE_FOLDER)
    print(f"{SENTINEL_POSTFIRE_FOLDER} has been successfully create")
else:
    print(f"{SENTINEL_POSTFIRE_FOLDER} already exist")

F:\\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\prefire already exist
F:\\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire already exist


In [17]:

download_sentinel_satellite_image(
    satellite_name='COPERNICUS/S2',
    situation='postfire',
    dataset=sumatera_only,
    scale=50,
    file_format='GeoTIFF',
    folder_path=SENTINEL_POSTFIRE_FOLDER,
    )
print("done")

# print(sentinel_prefire_url_list)
# print(sentinel_postfire_url_list)


data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\0
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\1
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\2
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\3
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\4
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\s

data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\68
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\69
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\70
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\71
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\72
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geot

['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\144
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\145
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\146
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\147
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\148
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B

F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\212
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\213
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\214
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\215
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\216
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2

['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\289
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\290
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\291
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\292
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\293
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B

...

['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7619
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7620
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7621
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7622
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7623
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exis

F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7681
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7682
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7683
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7684
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7685
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\senti

['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7742
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7743
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7744
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7745
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7746
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exis

['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7805
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7806
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7807
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7808
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exist
['B3', 'B2'] already exist
data already exist
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7809
['DELTA_NBR', 'B12'] already exist
['B8', 'B4'] already exis

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2e10199d44660beabf4488635531be08-4e1d63e22d9bea7f4ab16d50bdf1c8b7:getPixels downloaded
download data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/61c50864091ae8d9ee8cffaf444bbeb7-76150032edd68852075d2f850747caae:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/61c50864091ae8d9ee8cffaf444bbeb7-76150032edd68852075d2f850747caae:getPixels downloaded
F:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire\7835
download data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/11559fa37d378c2cc071f01c290fea58-aac0ba39fef6f546d96c8b578f49df6e:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/11559fa37d378c2cc071f01c290fea58-aac0ba39fef6f546d96c8b578f49df6e:getPixels downloaded
download data from https://earthengine.googleapis.com/v1alp